# Taller de Transformaciones Básicas en 2D

## Objetivo
Explorar los conceptos fundamentales de **transformaciones geométricas** (traslación, rotación y escala) utilizando matrices de transformación.
Aplicar estas transformaciones de manera estática y animada en función del tiempo para un objeto 2D.

## 1. Instalación Automática de Dependencias

In [1]:
import subprocess
import sys

# Instalar dependencias automáticamente
libraries = ['numpy', 'matplotlib', 'imageio', 'pillow']

for lib in libraries:
    try:
        __import__(lib)
        print(f"✓ {lib} ya está instalado")
    except ImportError:
        print(f"Installing {lib}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", lib, "-q"])
        print(f"✓ {lib} instalado correctamente")

print("\n✅ Todas las dependencias están listas!")

✓ numpy ya está instalado


✓ matplotlib ya está instalado
✓ imageio ya está instalado
Installing pillow...


✓ pillow instalado correctamente

✅ Todas las dependencias están listas!


## 2. Importar Librerías Necesarias

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import imageio
import os
import warnings
warnings.filterwarnings('ignore')

print("✓ Librerías importadas correctamente")

✓ Librerías importadas correctamente


## 3. Definir Funciones de Transformación (Matrices 3x3 para 2D Homogéneo)

Las transformaciones geométricas en 2D se pueden representar mediante matrices de $3 \times 3$ utilizando **coordenadas homogéneas** $(x, y, 1)$.

In [3]:
def translation_matrix(tx, ty):
    """Devuelve una matriz de traslación 2D."""
    return np.array([
        [1, 0, tx],
        [0, 1, ty],
        [0, 0, 1]
    ])

def rotation_matrix(angle_degrees):
    """Devuelve una matriz de rotación 2D (alrededor del origen)."""
    theta = np.radians(angle_degrees)
    return np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta),  np.cos(theta), 0],
        [0,             0,            1]
    ])

def scale_matrix(sx, sy):
    """Devuelve una matriz de escala 2D (con respecto al origen)."""
    return np.array([
        [sx, 0,  0],
        [0,  sy, 0],
        [0,  0,  1]
    ])

def apply_transform(points, matrix):
    """
    Aplica una matriz de transformación a un conjunto de puntos 2D.
    points format: array of shape (N, 2)
    """
    # Convertir a coordenadas homogéneas (x, y, 1)
    n_points = points.shape[0]
    homogenous_points = np.hstack([points, np.ones((n_points, 1))])
    
    # Multiplicar matriz por los puntos: (M * V^T)^T
    transformed_points = (matrix @ homogenous_points.T).T
    
    # Volver al espacio 2D
    return transformed_points[:, :2]


## 4. Crear la Figura Principal
Definiremos un polígono simple (por ejemplo, una letra o un cuadrado estilizado).

In [4]:
# Figura base: Un pequeño cohete/flecha apuntando arriba
base_shape = np.array([
    [0, 2],
    [-1, 0],
    [-0.5, 0],
    [-0.5, -1],
    [0.5, -1],
    [0.5, 0],
    [1, 0]
])

print("Forma base creada con", len(base_shape), "vértices.")

Forma base creada con 7 vértices.


## 5. Animación de Transformaciones en Función del Tiempo
Generaremos fotogramas animando la rotación, traslación (trayectoria circular/ovalada) y escala, combinándolas en cada paso de tiempo `t`.

In [5]:
print("🎬 Generando animación de transformaciones...")

os.makedirs("../media/python", exist_ok=True)
gif_path = '../media/python/transformaciones_2d.gif'

frames = []
num_frames = 60

for i in range(num_frames):
    t = i / float(num_frames) # t va de 0 a 1
    
    # 1. Rotación completa a lo largo del tiempo
    angle = t * 360  
    
    # 2. Escala oscilante (respiración)
    scale_factor = 1.0 + 0.5 * np.sin(t * 2 * np.pi * 2) # Oscila 2 veces
    
    # 3. Traslación en una trayectoria circular/ovalada
    # Radio de 5 unidades en X y 3 unidades en Y
    tx = 5 * np.cos(t * 2 * np.pi)
    ty = 3 * np.sin(t * 2 * np.pi)
    
    # Construir la matriz de transformación compuesta:
    # T_final = T_traslacion * T_rotacion * T_escala 
    # Ojo con el orden: primero escala, luego rota, luego traslada
    M_scale = scale_matrix(scale_factor, scale_factor)
    M_rot = rotation_matrix(angle)
    M_trans = translation_matrix(tx, ty)
    
    # Multiplicar de derecha a izquierda (la primera se aplica a la derecha)
    M_composed = M_trans @ M_rot @ M_scale
    
    # Aplicar la matriz
    transformed_shape = apply_transform(base_shape, M_composed)
    
    # --- Rendering ---
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_xlim(-10, 10)
    ax.set_ylim(-10, 10)
    ax.set_aspect('equal')
    ax.grid(True, linestyle='--', alpha=0.5)
    
    # Dibujar la trayectoria (opcional para contexto)
    path_t = np.linspace(0, 1, 100)
    path_x = 5 * np.cos(path_t * 2 * np.pi)
    path_y = 3 * np.sin(path_t * 2 * np.pi)
    ax.plot(path_x, path_y, 'k--', alpha=0.3)
    
    # Dibujar polígono
    poly = Polygon(transformed_shape, facecolor='cyan', edgecolor='blue', linewidth=2, alpha=0.8)
    ax.add_patch(poly)
    
    ax.set_title(f'Transformaciones (Frame {i+1}/{num_frames})\nT:({tx:.1f}, {ty:.1f}) R:{angle:.0f}° S:{scale_factor:.2f}x')
    
    # Convertir a imagen para GIF
    fig.canvas.draw()
    image_data = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
    image = image_data.reshape(fig.canvas.get_width_height()[::-1] + (4,))
    image = image[:, :, :3] # RGBA to RGB
    frames.append(image)
    
    plt.close(fig)

print(f"\n💾 Guardando GIF en {gif_path}...")
imageio.mimsave(gif_path, frames, duration=0.05)
print(f"✓ GIF creado exitosamente: {gif_path}")

🎬 Generando animación de transformaciones...



💾 Guardando GIF en ../media/python/transformaciones_2d.gif...


✓ GIF creado exitosamente: ../media/python/transformaciones_2d.gif
